In [21]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import numpy as np
view_log_train = pd.read_csv('../data/view_log.csv')
article_info = pd.read_csv('../data/article_info.csv')
submission = pd.read_csv('../submission/sample_submission.csv')

In [22]:
view_log_train.head(100)

,userID,articleID,userRegion,userCountry
0,USER_0000,ARTICLE_0661,NY,US
1,USER_0000,ARTICLE_2316,NY,US
2,USER_0000,ARTICLE_1345,NY,US
3,USER_0000,ARTICLE_1089,NY,US
4,USER_0000,ARTICLE_1484,NY,US
...,...,...,...,...
95,USER_0005,ARTICLE_0564,MG,BR
96,USER_0005,ARTICLE_0931,MG,BR
97,USER_0005,ARTICLE_2223,MG,BR
98,USER_0005,ARTICLE_1577,MG,BR


In [23]:
view_log_train.drop(['userRegion','userCountry'],axis =1 , inplace=True)
df = view_log_train

In [24]:
# SVD 모델 생성
svd = TruncatedSVD(n_components=50, random_state=42)
user_factors = svd.fit_transform(user_item_matrix)
item_factors = svd.components_.T

In [25]:
# 특정 사용자에 대한 추천 생성 함수
def get_svd_recommendations(user_id, n_recommendations=5):
    user_index = user_item_matrix.index.tolist().index(user_id)
    user_vector = user_factors[user_index]
    scores = user_vector.dot(item_factors.T)
    sorted_indices = np.argsort(scores)[::-1]
    recommendations = [user_item_matrix.columns[i] for i in sorted_indices if user_item_matrix.iloc[user_index, i] == 0][:n_recommendations]
    return recommendations

In [26]:
# 모든 사용자에 대해 추천 항목 생성
recommendations = []
for user in user_item_matrix.index:
    top5recommend = get_svd_recommendations(user, n_recommendations=5)
    for article in top5recommend:
        recommendations.append([user, article])

In [27]:
# 추천 결과를 데이터프레임으로 변환
top_recommendations = pd.DataFrame(recommendations, columns=['userID', 'articleID'])

# 제출 파일 생성
submission = pd.DataFrame(columns=['userID', 'articleID'])
submission['userID'] = top_recommendations['userID']
submission['articleID'] = top_recommendations['articleID']

# 파일 저장
# submission.to_csv('svd_submission.csv', index=False)

In [28]:
submission

,userID,articleID
0,USER_0000,ARTICLE_1568
1,USER_0000,ARTICLE_2720
2,USER_0000,ARTICLE_1948
3,USER_0000,ARTICLE_0561
4,USER_0000,ARTICLE_2147
...,...,...
7070,USER_1420,ARTICLE_0017
7071,USER_1420,ARTICLE_0436
7072,USER_1420,ARTICLE_0411
7073,USER_1420,ARTICLE_1089
